<a href="https://colab.research.google.com/github/bhagu/datascience/blob/main/Renewable%20Energy%20Forecasting/n_estimators_in_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import xgboost as xgb
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [5]:
#Load dataset
california_housing = fetch_california_housing()
california_housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [6]:
X_train, X_test, y_train, y_test = train_test_split(california_housing.data, california_housing.target, test_size=0.2, random_state=42)

In [7]:
# Initialize the model with a specific number of estimators
xg_reg = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, objective='reg:squarederror')

In [8]:
# Train the model
xg_reg.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [9]:
# Predict
y_pred = xg_reg.predict(X_test)

In [10]:
# Evaluate
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.29522676196268116


In [13]:
# Perform cross-validation with early stopping
cv_results = xgb.cv(
    dtrain=xgb.DMatrix(X_train, label=y_train),
    params={'objective': 'reg:squarederror', 'max_depth': 3, 'learning_rate': 0.1},
    nfold=5,
    num_boost_round=2000,  # Large number to ensure early stopping kicks in
    early_stopping_rounds=10,
    metrics='rmse',
    as_pandas=True,
    seed=42
)

In [14]:
# Get the number of boosting rounds that gave the best results
optimal_n_estimators = cv_results.shape[0]
print("Optimal number of estimators:", optimal_n_estimators)

Optimal number of estimators: 1303


In [15]:
# Train the model with the optimal number of estimators
xg_reg_optimal = xgb.XGBRegressor(n_estimators=optimal_n_estimators, learning_rate=0.1, max_depth=3, objective='reg:squarederror')
xg_reg_optimal.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1303, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [16]:
# Predict with the optimal model
y_pred_optimal = xg_reg_optimal.predict(X_test)

In [17]:
# Evaluate the optimal model
mse_optimal = mean_squared_error(y_test, y_pred_optimal)
print("Mean Squared Error with optimal n_estimators:", mse_optimal)

Mean Squared Error with optimal n_estimators: 0.21299623984646052
